
# Deep Learning Model Implementation for Binary Classification

This notebook implements a deep learning model for binary classification using a small numerical dataset. The model architecture will consist of dense layers with dropout and regularization to prevent overfitting.


In [85]:
# %conda install tensorflow
# Necessary imports
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l1_l2


In [86]:

# Load and preprocess the data
data = pd.read_csv('/home/matt/Projects/water-ml/datasets/labelled.csv')  # Update with actual path
data.replace('ND', 0, inplace=True)

# Encoding categorical features
categorical_columns = ['Taxa A1', 'Taxa A2', 'Taxa A3', 'Taxa A4', 'Taxa A5', 'Taxa B1', 'Taxa B2', 'Taxa B3']  # Update as needed
for col in categorical_columns:
    data[col] = data[col].astype(str)
    
encoder = OneHotEncoder(sparse=False)    
data_encoded = pd.DataFrame(encoder.fit_transform(data[categorical_columns]))
data_encoded.columns = encoder.get_feature_names_out(categorical_columns)
data.drop(categorical_columns ,axis=1, inplace=True)
data = pd.concat([data, data_encoded], axis=1)

# Separating features and target variable
X = data.drop(['Scheme', 'Sample'], axis=1)  # Update target and identifier columns as needed
y = data['Scheme'].map({'Stable': 0, 'Failure': 1})

# Normalizing the dataset
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


/home/matt/miniconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [87]:
from imblearn.over_sampling import SMOTE

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Augmenting the dataset with SMOTE
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

In [88]:

# Model architecture v1 (overfitting)
# model = Sequential()
# model.add(Dense(64, activation='relu', input_shape=(X_scaled.shape[1],), kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
# model.add(Dropout(0.5))
# model.add(Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='sigmoid'))

# Adjusted Model architecture
# model = Sequential()
# model.add(Dense(32, activation='relu', input_shape=(X_smote.shape[1],), kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
# # Reduced dropout rate
# model.add(Dropout(0.3))
# model.add(Dense(16, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
# # Removed one layer to simplify the model
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [89]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# # Adjusting the model architecture and parameters
# model = Sequential()
# model.add(Dense(16, activation='relu', input_shape=(X_smote.shape[1],), kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
# model.add(Dropout(0.2))
# model.add(Dense(8, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model with a possibly adjusted learning rate
# model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

# # Implementing Early Stopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# # Model training with Early Stopping
# history = model.fit(X_smote, y_smote, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=1)

In [90]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def create_model(l1_rate=0.001, l2_rate=0.001, learning_rate=0.001):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(X_smote.shape[1],), kernel_regularizer=l1_l2(l1=l1_rate, l2=l2_rate)))
    model.add(Dropout(0.3))
    model.add(Dense(16, activation='relu', kernel_regularizer=l1_l2(l1=l1_rate, l2=l2_rate)))
    model.add(Dense(1, activation='sigmoid'))
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [91]:
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

param_grid = {
    'l1_rate': [0.001, 0.01, 0.1],
    'l2_rate': [0.001, 0.01, 0.1],
    'learning_rate': [0.001, 0.01, 0.1]
}

/tmp/ipykernel_579234/3737891085.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)


In [92]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, scoring='accuracy')
grid_result = grid.fit(X_smote, y_smote)

2024-02-20 11:41:15.165021: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-20 11:41:15.165160: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-20 11:41:15.177411: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_B

3/3 [==============================] - 0s 2ms/step


3/3 [==============================] - 0s 1ms/step


3/3 [==============================] - 0s 2ms/step


3/3 [==============================] - 0s 994us/step


...

3/3 [==============================] - 0s 3ms/step


3/3 [==============================] - 0s 1ms/step


3/3 [==============================] - 0s 1ms/step


3/3 [==============================] - 0s 997us/step


3/3 [==============================] - 0s 770us/step


In [93]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: 0.873105 using {'l1_rate': 0.001, 'l2_rate': 0.1, 'learning_rate': 0.001}
0.849631 (0.086888) with: {'l1_rate': 0.001, 'l2_rate': 0.001, 'learning_rate': 0.001}
0.830718 (0.086475) with: {'l1_rate': 0.001, 'l2_rate': 0.001, 'learning_rate': 0.01}
0.428303 (0.209878) with: {'l1_rate': 0.001, 'l2_rate': 0.001, 'learning_rate': 0.1}
0.873038 (0.063764) with: {'l1_rate': 0.001, 'l2_rate': 0.01, 'learning_rate': 0.001}
0.844869 (0.091753) with: {'l1_rate': 0.001, 'l2_rate': 0.01, 'learning_rate': 0.01}
0.675453 (0.239631) with: {'l1_rate': 0.001, 'l2_rate': 0.01, 'learning_rate': 0.1}
0.873105 (0.069762) with: {'l1_rate': 0.001, 'l2_rate': 0.1, 'learning_rate': 0.001}
0.812005 (0.102789) with: {'l1_rate': 0.001, 'l2_rate': 0.1, 'learning_rate': 0.01}
0.320322 (0.068834) with: {'l1_rate': 0.001, 'l2_rate': 0.1, 'learning_rate': 0.1}
0.859021 (0.080312) with: {'l1_rate': 0.01, 'l2_rate': 0.001, 'learning_rate': 0.001}
0.830651 (0.071357) with: {'l1_rate': 0.01, 'l2_rate': 0.001, 'learni